In [1]:
import pandas as pd

In [28]:
# 1. Read the CSV file into a DataFrame
df = pd.read_csv('DSNY_Monthly_Tonnage_Data.csv')

In [29]:
# Convert the columns to integer type
df['BOROUGH_ID'] = df['BOROUGH_ID'].astype(int)
df['COMMUNITYDISTRICT'] = df['COMMUNITYDISTRICT'].astype(int)

# 2. Create a new column 'DSTRICT'
df['DISTRICT'] = df['BOROUGH_ID'].astype(str) + df['COMMUNITYDISTRICT'].apply(lambda x: f'{x:02d}')


In [30]:
import geopandas as gpd

In [31]:
# 2. Read the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file('DSNY Districts_20231015.geojson')

In [32]:
gdf.head()

,districtcode,fid,shape_area,globalid,shape_length,district,geometry
0,503,111,0.005915098995203206,{39F3B701-E1E4-4D24-9E7A-D12AF02CD8DF},0.5756654155520395,SI03,"MULTIPOLYGON (((-74.20073 40.57933, -74.19930 ..."
1,502,107,0.005862061928047071,{08F7FCC6-15B3-496C-B81F-E515D37599F2},0.4580115697976364,SI02,"MULTIPOLYGON (((-74.19569 40.63503, -74.19558 ..."
2,501,105,0.0037197572830243385,{34CCF947-B803-47E3-A9B1-4173C3D3B2A4},0.5014572369711472,SI01,"MULTIPOLYGON (((-74.08221 40.64829, -74.08213 ..."
3,210,84,0.0017178920314047566,{CBCD9507-B7A3-41C7-BD53-2769B5D3775C},0.7757034908943723,BX10,"MULTIPOLYGON (((-73.78282 40.83635, -73.78303 ..."
4,202,81,0.0006042251770444894,{C6434E59-02B1-4ADA-9473-68A2C4517BE9},0.13663982240687578,BX02,"MULTIPOLYGON (((-73.89386 40.82735, -73.89293 ..."


In [33]:
# combine data by year

In [34]:
# 2. Extract the year from the "MONTH" column
df['YEAR'] = df['MONTH'].str.split('/').str[0]

In [35]:
# 3. Group by the extracted year and sum the relevant columns
cols_to_sum = [
    "REFUSETONSCOLLECTED", "PAPERTONSCOLLECTED", "MGPTONSCOLLECTED", 
    "RESORGANICSTONS", "SCHOOLORGANICTONS", "LEAVESORGANICTONS", "XMASTREETONS"
]

In [47]:
# If you want to group by year AND district, you can replace ['YEAR'] with ['YEAR', 'DSTRICT'] in the groupby.
aggregated = df.groupby(['YEAR','DISTRICT','BOROUGH'])[cols_to_sum].sum().reset_index()

In [48]:
aggregated.tail(5)

,YEAR,DISTRICT,BOROUGH,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,SCHOOLORGANICTONS,LEAVESORGANICTONS,XMASTREETONS
1964,2023,413,Queens,48117.8,3790.7,7349.2,985.3,0.0,0.0,25.0
1965,2023,414,Queens,28350.9,2100.2,2269.1,468.6,0.0,0.0,13.5
1966,2023,501,Staten Island,48396.1,4876.2,5905.0,0.0,0.0,0.0,26.5
1967,2023,502,Staten Island,38116.5,4260.0,4475.6,0.0,175.6,0.0,18.6
1968,2023,503,Staten Island,50642.1,6041.6,5907.4,0.0,190.6,0.0,27.2


In [50]:
aggregatedGeo = pd.merge(aggregated, gdf[['districtcode', 'geometry']], left_on='DISTRICT', right_on='districtcode', how='left')

In [51]:
aggregatedGeo.head(5)

,YEAR,DISTRICT,BOROUGH,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,SCHOOLORGANICTONS,LEAVESORGANICTONS,XMASTREETONS,districtcode,geometry
0,1990,101,Manhattan,1815.9,0.0,0.0,0.0,0.0,0.0,0.0,101,"MULTIPOLYGON (((-74.01114 40.72582, -74.01103 ..."
1,1990,102,Manhattan,7447.3,0.0,0.0,0.0,0.0,0.0,0.0,102,"MULTIPOLYGON (((-73.99684 40.73737, -73.99362 ..."
2,1990,103,Manhattan,17926.8,0.0,0.0,0.0,0.0,0.0,0.0,103,"MULTIPOLYGON (((-73.98878 40.73397, -73.98718 ..."
3,1990,104,Manhattan,5218.0,0.0,0.0,0.0,0.0,0.0,0.0,104,"MULTIPOLYGON (((-73.99394 40.77319, -73.99370 ..."
4,1990,105,Manhattan,6801.9,0.0,0.0,0.0,0.0,0.0,0.0,105,"MULTIPOLYGON (((-73.97302 40.76429, -73.97141 ..."


In [52]:
aggregatedGeo.to_csv('aggregated_Tonnage_and_Geodata_byYear.csv', index=False)

# Convert the merged DataFrame to a GeoDataFrame
gdf_aggregatedGeo = gpd.GeoDataFrame(aggregatedGeo, geometry='geometry')

# Save the GeoDataFrame as a GeoJSON file
gdf_aggregatedGeo.to_file('aggregated_Tonnage_and_Geodata_byYear.geojson', driver='GeoJSON')